# Exploratory data analysis

### Chapters:
* [How to import our dataset](#How-to-import-our-dataset)
* [Summarize our dataset](#Summarize-our-dataset)
* [Slicing and subsetting our dataset](#Slicing-and-subsetting-our-dataset)
* [How to plot using Pandas](#How-to-plot-using-Pandas)

## How to import our dataset

Ourdata set is already prepared in a csv format.

In [ ]:
import os
import glob
from ast import literal_eval
import json
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline  

In [ ]:
BASE = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(__name__))))

In [ ]:
folder = os.path.join(BASE, "data/cleaned")

In [ ]:
files = glob.glob(os.path.join(folder, "*/*.csv"))

In [ ]:
files[0:2]

In [ ]:
len(files)

In [ ]:
dataset = pd.read_csv(files[0], nrows=10)

A DataFrame is a 2-dimensional data structure that can store data of different types (including characters, integers, floating point values, factors and more) in columns. 

It is similar to a spreadsheet or an SQL table or the data.frame in R.

A DataFrame always has an index (0-based).

An index refers to the position of an element in the data structure.

In [ ]:
# Our dataset
dataset.head(2)

With `head()` with can show the first x rows of our dataframe, by default 5.

**Do you see the indexes and columns?**

Let’s look at series' types.

In [ ]:
dataset.info()

With `info()` we get useful information about our dataset, like size, number of rows, number of columns and types.

**For example we see that dates have the wrong type (object instead of datatime). We have to resolve this later.**

In [ ]:
dataset['company'].dtype

In [ ]:
dataset.dtypes

## Summarize our dataset

In [ ]:
dataset.describe()

`describe()` gives us important stats about our dataset.

**In this specific case they are meaningless, given the context.**

Let's import all our csv files.

This time parsing dates.

In [ ]:
dataframes = [pd.read_csv(fp, parse_dates=['published_on', 'updated_on']) for fp in files]
df = pd.concat(dataframes, ignore_index=True)

In [ ]:
df.shape

In [ ]:
df.tail(2)

## Series & Dataframes

In [ ]:
df.info()

Each column is a series, together they create a dataframe.

A dataframe can have columns of different type, a series can have only 1 type, see the `df.info`

In [ ]:
type(df), type(df['company'])

**Series are based on numpy arrays:**

In [ ]:
df['id'].values

In [ ]:
type(df['id'].values)

In [ ]:
df.columns

## Slicing and subsetting our dataset

### Let's start cleaning.

Let's check if we have have columns with non-unique values

In [ ]:
df.nunique()

In [ ]:
df.head(2)

In [ ]:
df[df['id'] == 68396].shape

<div class="alert alert-success">
    <b>Exercise:</b> could you find another id that is not unique?
</div>

This is how we can `select` a specific column:

In [ ]:
df[["link"]].head()

<div class="alert alert-warning">
    <b>I use `head()` just to limit the output of the command (5 rows).</b>
</div>

<div class="alert alert-success">
    <b>Exercise:</b> What is the difference between `df[["link"]].head()` and `df["link"].head()`?
</div>

Selecting more than one columns follows the same command pattern:

In [ ]:
df[["company", "link"]].head()

<div class="alert alert-success">
    <b>Exercise:</b> Try to play with the command and select different columns
</div>

With this syntax you can select a subset of you dataframe, you can think about the `where` condition in SQL.

In [ ]:
df[df["company"] == "MEDWING"].head()

<div class="alert alert-success">
    <b>Exercise:</b> Can select the rows of a different company?
</div>

It' now time to remove all the duplicates.

In [ ]:
df.id.nunique()

In [ ]:
df = df.drop_duplicates(['id'])

In [ ]:
df[df['id'] == 68396].shape

We can also slice and subset ussing `loc`, label based indexing, and `iloc`, integer based indexing.

<div class="alert alert-success">
    <b>Exercise:</b> What is the shape of the entire dataframe now?
</div>

In [ ]:
df.loc[:, ["slug", "title"]].head()

In [ ]:
df.iloc[10:15, 7:9]

Tags are really hard to use in this way. They are not really [tidy](http://vita.had.co.nz/papers/tidy-data.html).

In [ ]:
df['tags'].index

In [ ]:
%%time
df['tags'] = pd.Series([literal_eval(x[1]) for x in df['tags'].iteritems()], index=df['tags'].index)

In [ ]:
tags = df['tags'].apply(pd.Series)

In [ ]:
tags.head()

In [ ]:
tags.tail()

We just created a new dataframe, with columns expanded.

<div class="alert alert-warning">
    <b>Why do we have some many NaN and 43 columns?</b>
</div>

In [ ]:
tags = tags.rename(columns = lambda x : 'tag_' + str(x))

In [ ]:
tags.head()

In [ ]:
len(tags)

Let's put this dataframes together.

In [ ]:
df_1 = pd.concat([df, tags], axis=1)

### What is the axis?

![alt text](../../images/python-operations-across-axes.png "Axes Pandas")

In [ ]:
len(df_1['id'])

In [ ]:
df_1.head(2)

In [ ]:
df_2 = pd.wide_to_long(df_1, stubnames='tag_', i='id', j='tags_')

In [ ]:
df_2 = df_2.drop(labels="tags", axis=1)

In [ ]:
df_2.head(2)

In [ ]:
len(df_2)

In [ ]:
df_2.loc[46098][:5]

In [ ]:
df_2.info()

In [ ]:
df_2 = df_2.reset_index()

In [ ]:
df_2.head(1)

In [ ]:
df_2['tags_'] = df_2['tags_'].astype(np.uint16)

In [ ]:
df_2.head(1)

In [ ]:
df_2 = df_2.rename(columns={"tags_": "tag_order", "tag_": "tag"})

In [ ]:
df_2.head(1)

We filter, taking rows, where:
    * tag_order > 0
    * tag is not NaN

In [ ]:
df_2 = df_2[~((df_2['tag'].isnull()) & (df_2['tag_order'] > 0))]

In [ ]:
df_2.head()

In [ ]:
unique_tags = df_2[df_2["tag"].notnull()]["tag"].unique()

In [ ]:
len(unique_tags)

In [ ]:
unique_tag_id = list(filter(lambda x: isinstance(x, np.float), unique_tags))

In [ ]:
len(unique_tag_id)

In [ ]:
# Pull tags from Berlinstartupjobs

from functools import partial
import time
from tornado import ioloop
from tornado import gen
from tornado import httpclient
import json

tags_to_name = {}
tags_file = os.path.join(BASE, "data/cleaned/berlinstartupjobs/tags/tags_list.csv")

@gen.coroutine
def fetcher(tags):
    """Call the api endpoint to get the name of each tag."""
    
    now = time.time()
    
    print(f"Fetching {len(tags)} tags.......")
    
    @gen.coroutine
    def async_client(url):

        try:
            response = yield httpclient.AsyncHTTPClient().fetch(url)
        except httpclient.HTTPError as err:
            # HTTPError is raised for non-200 responses; the response
            # can be found in e.response.
            print("Error: " + str(err))
        return response
    
    print("Fetched tag...", end="")
    
    for tag in tags:
        _tag = int(tag)
        url = f"http://berlinstartupjobs.com/wp-json/wp/v2/tags/{_tag}"
        
        response = yield async_client(url)
        
        json_body = json.loads(response.body)
        tags_to_name[_tag] = json_body['slug']
        
        print(f"{_tag}..", end="")     
    
    duration = time.time() - now
    print(f"\nTags fetched in {duration} secs")

if False:
    io_loop = ioloop.IOLoop.current()
    io_loop.add_callback(partial(fetcher, tags=unique_tag_id))
    df_tags = pd.DataFrame.from_dict(tags_to_name, orient='index')
    df_tags = df_tags.reset_index()
    df_tags.columns = ["tag_id", "tag_name"]
    df_tags.to_csv(tags_file, index=False)

In [ ]:
df_tags = pd.read_csv(tags_file, index_col=["tag_id"])
tags_to_name = df_tags.to_dict(orient='dict')['tag_name']

In [ ]:
# Change tags from number to word
def change_value(x):
    value = tags_to_name.get(x)
    if value is not None:
        return value
    return x    

In [ ]:
df_2['tag'] = df_2['tag'].apply(change_value)

In [ ]:
df_2.head(1)

In [ ]:
df_2.info()

We can verify again that rows with tag == NaN are unique.

In [ ]:
df_2[df_2["tag"].isnull()]

Pandas comes with a very useful collection of functions, they are very **fast**.

<div class="alert alert-success">
    <b>Exercise:</b> How can you show the rows that contains "python"?
</div>

In [ ]:
df_2.slug.str.contains("python", case=False)

<div class="alert alert-success">
    <b>Exercise:</b> Check the company GetYourGuide, what's the problem?
</div>

In [ ]:
df_2.loc[df_2.company.str.contains("getyour", case=False)]

In [ ]:
df_2.loc[df_2.company.str.contains("getyour", case=False), "company"] = "GetYourGuide"

Time to remove `tag_order` column.

In [ ]:
df_2 = df_2.drop(labels="tag_order", axis=1)

In [ ]:
df_2.head(1)

## How to plot using Pandas

We count how many times each tag is used:

In [ ]:
tags_counting = df_2['tag'].value_counts()

In [ ]:
tags_counting.describe()

Number of tags used less than 20 times

In [ ]:
less_commonot_tags = tags_counting[tags_counting < 10]
len(less_commonot_tags)

Most used tags

In [ ]:
most_commonot_tags = tags_counting[tags_counting >= 40]
len(most_commonot_tags)

In [ ]:
most_commonot_tags.plot(kind='barh', figsize=(10,10), fontsize=15, title="Tags")

Now want to consider the each job offer, without considering tags, so they must be unique.

In [ ]:
grouped = df_2.drop(["tag"], axis=1)
grouped = grouped.drop_duplicates()

In [ ]:
sorted_number_offers = grouped.groupby("company")['id'].count().sort_values(ascending=False)

In [ ]:
# first 10 companies with most job offers
sorted_number_offers[:10].plot(kind='barh', figsize=(10,10), fontsize=15)

In [ ]:
# last 10 companies per number of job offers
sorted_number_offers[-10:].plot(kind='barh', figsize=(10,10), title="Job offers by companies", fontsize=15)

In [ ]:
grouped['weekday'] = grouped['published_on'].dt.dayofweek

In [ ]:
grouped.head()

In [ ]:
grouped_by_day = grouped.groupby("weekday")['id'].count()

In [ ]:
grouped_by_day

In [ ]:
grouped_by_day.index = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [ ]:
grouped_by_day.plot(kind='bar', figsize=(10,10), title="Offer published during the week", fontsize=15)

In [ ]:
grouped.set_index("published_on").loc[:,["id"]].resample('D', label="published_on").count().plot(kind="bar", figsize=(20,10))

<div class="alert alert-success">
    <b>Exercise:</b> Can you find the most Pythonic companies?
</div>

In [ ]:
python_offers = df_2.loc[df_2.content.str.contains("python", case=False)]
pythonic_companies = python_offers.groupby("company")[["id"]].count()
pythonic_companies[pythonic_companies["id"] >10].sort_values(by="id", ascending=False).plot(kind="bar")

In [ ]:
java_offers = df_2.loc[df_2.content.str.contains("java", case=False)]
java_companies = java_offers.groupby("company")[["id"]].count()
java_companies[java_companies["id"] >15].sort_values(by="id", ascending=False).plot(kind="bar")

In [ ]:
data_offers = grouped.loc[df_2.title.str.contains("data", case=False)]
data_companies = data_offers.groupby("company")[["id"]].count()
data_companies[data_companies["id"] >=2].sort_values(by="id", ascending=False).plot(kind="bar")

In [ ]:
grouped.loc[grouped.title.str.contains("data", case=False)][["id", "company", "title" ]].head()